## Libs

In [3]:
import pandas as pd
import numpy as np
import pickle 

import seaborn as sns
import matplotlib.pylab as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score

from yellowbrick.classifier import confusion_matrix
from yellowbrick.classifier import ROCAUC

from lazypredict.Supervised import LazyClassifier